In [7]:
import numpy as np
from hmm_utils import HMM
from attackers.smoothing_state_attacker import ss_attacker
from attackers.decoding_attacker import dec_attacker
from params import *
from solvers.mc_ennumeration import MC_enumeration
#from solvers.simulated_annealing import simulated_annealing
#from solvers.aps_gibbs import aps_gibbs
from solvers.nn_RS.nn_RS import *
#from solvers.aps_gibbs_class import *

from solvers.aps_gibbs_class_exp_decay import *
from solvers.random_mc_ennumeration import *

from scipy.special import softmax

In [8]:
priors     = np.array([0.5,0.3,0.2])
transition = np.array([[0.85, 0.05,0.1],
                       [0.05, 0.9,0.05],
                        [1/2, 1/4, 1/4]])
emission   = np.array([[0.699, 0.05, 0.1, 0.05, 0.1, 0.001],
                    [0.001, 0.1, 0.1, 0.299, 0.3, 0.2],
                       [0.1, 0.2, 0.1, 0.2, 0.1, 0.3]])

rho_probs = 0.95*np.ones(6)

x_vector = np.array([ [4], [3], [5], [3], [4] ])

In [9]:
hmm = HMM(n_components=3, n_obs=6 )
hmm.emissionprob_ = emission
hmm.transmat_     = transition
hmm.startprob_    = priors

In [10]:
V, seq = hmm.nu(x_vector)
seq

array([1, 1, 1, 1, 1])

In [11]:
attacker = dec_attacker(priors, transition, emission, rho_probs,
         x_vector, w1 = 100.0 ,w2 = 0.0 , seq = np.array([2,1,0,0,0]), k_value=10000)


# RS with Neural Network

In [12]:
from solvers.nn_RS.nn_RS import *
rs = nn_RS(attacker, flag='SA', RS_iters=5000, mcts_iters=50, sa_iters=100, eps=0.01, lr=0.1)
deltaT = 5

for t in np.arange(5, 100, deltaT):
    
    z_star, quality = rs.iterate(simulation_seconds = deltaT)
    print("Quality of best solution at time {:d} is {:f}".format(t, quality))
    

Quality of best solution at time 5 is 621.093977
Quality of best solution at time 10 is 621.093977
Quality of best solution at time 15 is 621.098118
Quality of best solution at time 20 is 621.139985
Quality of best solution at time 25 is 621.209158
Quality of best solution at time 30 is 621.209158
Quality of best solution at time 35 is 622.549641
Quality of best solution at time 40 is 622.549641
Quality of best solution at time 45 is 622.549641
Quality of best solution at time 50 is 702.150615
Quality of best solution at time 55 is 702.150615
Quality of best solution at time 60 is 702.467004
Quality of best solution at time 65 is 702.467004
Quality of best solution at time 70 is 713.865544
Quality of best solution at time 75 is 722.296585
Quality of best solution at time 80 is 722.296585
Quality of best solution at time 85 is 722.296585
Quality of best solution at time 90 is 722.296585
Quality of best solution at time 95 is 751.288633


In [14]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([2, 0, 0, 1, 1])